In [9]:
import json
import datetime
import pandas as pd
import math
import time
import requests
from requests.auth import HTTPBasicAuth


class ApiRequest:
    def __init__(self):
        self.base_url = "https://api.eia.gov/v2/"
        self.data_endpoint = "electricity/rto/fuel-type-data/data"
        self.api_key =  "pCPxJdvTkhxchamZtC3UDvC8b8tgd7vox18Vtj32"
        
    

    def generate_params(self, offset, start_date, end_date):
        params = {
            "frequency": "hourly",
            "data[0]": "value",
            
#             "facets[respondent][0]": "ERCO",
#             "facets[respondent][1]": "ERCO",
#             "facets[fueltype][0]":"COL",
#             "facets[fueltype][1]":"NG",
#             "facets[fueltype][2]":"NUC",
#             "facets[fueltype][4]":"SUN",
#             "facets[fueltype][5]":"WAT",
#             "facets[fueltype][6]":"WND",
#             "facets[fueltype][3]":"OTH",
         
            "start": start_date,
            "end": end_date,
            "sort[0][column]": "period",
            "sort[0][direction]": "asc",
            "offset": offset,
            "length": 5000,
            "api_key": self.api_key
        }

        return params

    def get_with_pagination(self):


        # initializing date
        test_date = datetime.datetime.strptime("2023-01-01", "%Y-%m-%d")

        # initializing K 
        K = 365

        date_generated = pd.date_range(test_date, periods=K)
        print(date_generated.strftime("%Y-%m-%d"))
        li=date_generated.strftime("%Y-%m-%d")  
        
        
        
        
        maindata=[]
        
        for i in li:
            print(i)
            
            start_date=i+"T00"
            end_date=i+"T07"
            parameters = self.generate_params(offset=0, start_date=start_date, end_date=end_date)
            response = requests.get(url=self.base_url + self.data_endpoint, params=parameters)
            response_json = response.json()
            total_data_length = int(response_json["response"]["total"])
            
            print(total_data_length)
            
            offset = 0
            limit = 5000

            actual_data = response_json["response"]["data"]
#             print(actual_data)
            maindata=maindata+actual_data
            
            start_date=i+"T08"
            end_date=i+"T15"
            parameters = self.generate_params(offset=0, start_date=start_date, end_date=end_date)
            response = requests.get(url=self.base_url + self.data_endpoint, params=parameters)
            time.sleep(2)
            response_json = response.json()
            total_data_length = int(response_json["response"]["total"])
            
#             print(total_data_length)
            
            offset = 0
            limit = 5000

            actual_data = response_json["response"]["data"]
#             print(actual_data)
            maindata=maindata+actual_data
            
            start_date=i+"T16"
            end_date=i+"T23"
            parameters = self.generate_params(offset=0, start_date=start_date, end_date=end_date)
            response = requests.get(url=self.base_url + self.data_endpoint, params=parameters)
            response_json = response.json()
            total_data_length = int(response_json["response"]["total"])
            
#             print(total_data_length)
            
            offset = 0
            limit = 5000

            actual_data = response_json["response"]["data"]
#             print(actual_data)
            maindata=maindata+actual_data


        return maindata
#             return actual_data




call = ApiRequest()
data=call.get_with_pagination()

Index(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05',
       '2023-01-06', '2023-01-07', '2023-01-08', '2023-01-09', '2023-01-10',
       ...
       '2023-12-22', '2023-12-23', '2023-12-24', '2023-12-25', '2023-12-26',
       '2023-12-27', '2023-12-28', '2023-12-29', '2023-12-30', '2023-12-31'],
      dtype='object', length=365)
2023-01-01
3104
2023-01-02
3104
2023-01-03
3104
2023-01-04
3104
2023-01-05
3104
2023-01-06
3104
2023-01-07
3104
2023-01-08
3104
2023-01-09
3104
2023-01-10
3104
2023-01-11
3104
2023-01-12
3104
2023-01-13
3104
2023-01-14
3104
2023-01-15
3104
2023-01-16
3104
2023-01-17
3104
2023-01-18
3104
2023-01-19
3104
2023-01-20
3104
2023-01-21
3104
2023-01-22
3104
2023-01-23
3104
2023-01-24
3104
2023-01-25
3104
2023-01-26
3106
2023-01-27
3110
2023-01-28
3104
2023-01-29
3104
2023-01-30
3104
2023-01-31
3104
2023-02-01
3104
2023-02-02
3104
2023-02-03
3104
2023-02-04
3104
2023-02-05
3104
2023-02-06
3104
2023-02-07
3104
2023-02-08
3104
2023-02-09
3104
2023-0

In [14]:
import pandas as pd
import numpy as np
df = pd.DataFrame(data)
pivot_df = df.pivot_table(index=['period', 'respondent'], columns='type-name', values='value', aggfunc='first', fill_value=np.nan)
 

pivot_df.reset_index(inplace=True)

# # If you want to fill NaN values with 0
# pivot_df.fillna(0, inplace=True)

# Display the result
# print(pivot_df)
pivot_df.columns = pivot_df.columns.str.upper()
# columns = list(pivot_df.columns)
# desired_order = ['PERIOD', 'NATURAL GAS', 'COAL', 'NUCLEAR', 'SOLAR', 'WIND', 'HYDRO', 'OTHER']
# for i in desired_order:
#     if i not in columns:
#         pivot_df[i] = None
# pivot_df= pivot_df[desired_order]
# pivot_df['RESPONDENT']=df['respondent']
pivot_df['VALUE-UNITS']=df['value-units']

pivot_df["PERIOD"] = pd.to_datetime(pivot_df["PERIOD"] , format="%Y-%m-%dT%H")
pivot_df.rename(columns={"PERIOD": "timestamp"}, inplace=True)
pivot_df.rename(columns={"RESPONDENT": "BALANCING REGION"}, inplace=True)
pivot_df=pivot_df.reset_index(drop=True)

# pivot_df.to_csv(pivot_df['RESPONDENT'][0]+'.csv',index=False)
pivot_df.to_csv('Balancing_Regions_Final.csv',index=False)
# pivot_df= pivot_df.drop('type-name', axis=1)
print('Scraper done!')

Scraper done!


In [1]:
weather = pd.read_csv('Austin.csv')

NameError: name 'pd' is not defined

In [10]:
from datetime import datetime
last_column = weather.pop('timestamp') 
  
# insert column using insert(position,column_name, 
# first_column) function 
weather.insert(0, 'timestamp', last_column) 
for i in range (len(weather.index)):
    datetime_string = weather['timestamp'][i]
    

# Parse the string into a datetime object
    original_datetime = datetime.strptime(datetime_string, '%Y-%m-%d %H:%M:%S')

# Replace the hour part with 0
    new_datetime = original_datetime.replace(minute=0)
    weather['timestamp'][i]=new_datetime
    


In [8]:
data

NameError: name 'data' is not defined

In [12]:
data

[{'period': '2023-01-01T00',
  'respondent': 'PNM',
  'respondent-name': 'Public Service Company of New Mexico',
  'fueltype': 'SUN',
  'type-name': 'Solar',
  'value': 25,
  'value-units': 'megawatthours'},
 {'period': '2023-01-01T00',
  'respondent': 'PGE',
  'respondent-name': 'Portland General Electric Company',
  'fueltype': 'WND',
  'type-name': 'Wind',
  'value': 0,
  'value-units': 'megawatthours'},
 {'period': '2023-01-01T00',
  'respondent': 'GVL',
  'respondent-name': 'Gainesville Regional Utilities',
  'fueltype': 'COL',
  'type-name': 'Coal',
  'value': 0,
  'value-units': 'megawatthours'},
 {'period': '2023-01-01T00',
  'respondent': 'TEX',
  'respondent-name': 'Texas',
  'fueltype': 'OTH',
  'type-name': 'Other',
  'value': 305,
  'value-units': 'megawatthours'},
 {'period': '2023-01-01T00',
  'respondent': 'PACE',
  'respondent-name': 'PacifiCorp East',
  'fueltype': 'WAT',
  'type-name': 'Hydro',
  'value': 48,
  'value-units': 'megawatthours'},
 {'period': '2023-01-01